In [60]:
#IMPORTAMOS LAS LIBRERIAS BASADAS EN TENSORFLOW Y KERAS
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K

#LIMPIAMOS SESIN DE KERAS PARA EMPEZAR DESDE 0 EN MEMORIA
K.clear_session()
#GENERAMOS 2 VARIABLES CON CADA UNA DE LAS CARPETAS TANTO DE ENTRENAMIENTO COMO DE TEST CON FOTOGRAFIAS
data_entrenamiento = './CarneDataset/train'
data_validacion = './CarneDataset/test'
#DECLARACION DE VARIABLES CON VALORES QUE PARAMETRIZARAN EL DESEMPEÑO DEL ENTRENAMIENTO DEL MODELO DE CLASIFICACION DE FOTOS
epocas=15 #NUMERO DE EPOCHS PARA EL TRAINING
longitud, altura = 100, 100 #REDIMENSIONAMIENTO DE FOTOGRAFIAS
batch_size = 10 # TAMAÑO DE FOTOGRAFIAS A GESTIONAR
pasos = 120 #NUMERO DE STEPS PARA ENTRENAMIENTO
validation_steps = 80 #NUMERO DE STEPS PARA VALIDACION
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 8 #CANTIDAD DE CLASES DE FOTOGRAFIAS A CLASIFICAR
lr = 0.0004 #LERANING RATE PARA EL PROCESO DE ENTRENAMIENTO

In [61]:
#FUNCION IMAGEDATAGENERATOR QUE PROPORCIONARA MODIFICACION A CIERTAS FOTOGRAFIAS PARA OPTIMIZAR EL PROCESO DE ENTRENAMIENTO
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)
#FUNCION PARA CONFIGURACION DEL PROCESO DE RECUPERACION DE INFORMACION DE FOTOS PARA ENTRENAMIENTO
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')
#FUNCION PARA CONFIGURACION DEL PROCESO DE RECUPERACION DE INFORMACION DE FOTOS PARA TEST
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 1633 images belonging to 8 classes.
Found 810 images belonging to 8 classes.


In [62]:
#FUNCION PARA EMPEZAR EL MODELO Y SU CONSTRUCCION
cnn = Sequential()
#PRIMERA CAPA
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#SEGUNDA CAPA
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#RECONFIGURACION DE LA INFORMACION EN DATOS RECONOCIBLES POR EL MODELO
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))
#COMPILACION DE LA INFORMACION DENTRO DEL MODELO PARA SU ENTRENAMIENTO 
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])

In [63]:
#ENTRENAMIENTO DEL MODELO SEGUN LOS DATOS INGRESADOS COMO PARAMETROS
cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Epoch 1/15
120/120 [==============================] - 46s 379ms/step - loss: 1.6260 - accuracy: 0.5532 - val_loss: 1.1597 - val_accuracy: 0.5825
Epoch 2/15
120/120 [==============================] - 51s 423ms/step - loss: 1.0702 - accuracy: 0.6317 - val_loss: 1.1285 - val_accuracy: 0.6300
Epoch 3/15
120/120 [==============================] - 48s 402ms/step - loss: 0.8432 - accuracy: 0.7025 - val_loss: 0.6807 - val_accuracy: 0.7700
Epoch 4/15
120/120 [==============================] - 45s 377ms/step - loss: 0.7258 - accuracy: 0.7485 - val_loss: 1.3052 - val_accuracy: 0.4963
Epoch 5/15
120/120 [==============================] - 48s 398ms/step - loss: 0.6338 - accuracy: 0.7779 - val_loss: 0.9378 - val_accuracy: 0.6825
Epoch 6/15
120/120 [==============================] - 48s 400ms/step - loss: 0.6393 - accuracy: 0.7712 - val_loss: 1.0041 - val_accuracy: 0.6200
Epoch 7/15
120/120 [==============================] - 49s 405ms/step - loss: 0.5510 - accuracy: 0.7921 - val_loss: 0.6091 - val_ac

In [64]:
#CREACION Y ALMACENAMIENTO DEL RESULTANTE DEL ENTRENAMIENTO PARA RECONOCIMIENTO DE FOTOGRAFIAS
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')